In [128]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from IntelligentBuildingPerformanceDesign.AIBPD.data.database import Database
from IntelligentBuildingPerformanceDesign.AIBPD.data.preprocessing import PreprocessingCBECS
import pandas as pd
import math
def data_preprocess_BN_Designer(similarBldDF):
		'''
		'''
		coolLoadList=[]
		coolLoadList=[]
		equalSizeBin2=[1,2]
		equalSizeBin4=[1,2,3,4]
		equalSizeBin5=[1,2,3,4,5]
		similarBldDF['principleActivityN']=pd.qcut(similarBldDF['principleActivity'],5,labels=equalSizeBin5)
		#similarBldDF['principleActivity']#np.where(similarBldDF['principleActivity']==1, 1, 0)
		similarBldDF['climateZoneN']=pd.cut(similarBldDF['climateZone'],[0,1,2,3,7],labels=[1,2,3,4])
		similarBldDF['CDD65N']=pd.qcut(similarBldDF['CDD65'],5,labels=equalSizeBin5)
		similarBldDF['COOLPN']=pd.cut(similarBldDF['COOLP'],[10,90,100],labels=equalSizeBin2)
		similarBldDF['MAINCLN']=similarBldDF['MAINCL']
		similarBldDF['MAINCLN'][np.abs(similarBldDF['MAINCLN'])<1]=0
		similarBldDF['MAINCLN'][np.abs(similarBldDF['MAINCLN'])>3]=4
		similarBldDF=similarBldDF.dropna()
		for index, row in similarBldDF.iterrows():
			if row['climateZone']==1:
				coolLoadList.append(int(math.log(row['buildingArea']*0.8)))
			elif row['climateZone']==2 or row['climateZone']==5 or row['climateZone']==7:
				coolLoadList.append(int(math.log(row['buildingArea']*1.0)))
			elif row['climateZone']==3:
				coolLoadList.append(int(math.log(row['buildingArea']*1.2)))
			else:
				coolLoadList.append(int(math.log(row['buildingArea']*1.0)))
		similarBldDF['COOLLOAD']=coolLoadList
		del coolLoadList
		return similarBldDF

database=Database()
CBECS2012DF=database.select('CBECS2012')
preprocess=PreprocessingCBECS()
CBECS2012DF=CBECS2012DF[CBECS2012DF['CDD65']>500]
CBECS2012DF=preprocess.forHECLClf(CBECS2012DF)
candidateDF=data_preprocess_BN_Designer(CBECS2012DF)
candidateDF=candidateDF.dropna()
m,n=candidateDF.shape
m09=int(0.9*m)
m8=int(0.8*m)
#model for test
clf_test=DecisionTreeClassifier()
#proposed method
X_train_new=candidateDF[:m09][candidateDF['HECS']<=1][['climateZoneN','COOLLOAD','principleActivityN','MAINCLN']].values
y_train_new=candidateDF[:m09][candidateDF['HECS']<=1][['HECS']].values
X_test_new=candidateDF[m09:][candidateDF['HECS']<=1][['climateZoneN','COOLLOAD','principleActivityN','MAINCLN']].values
y_test_new=candidateDF[m09:][candidateDF['HECS']<=1][['HECS']].values

clf_new=MultinomialNB()
clf_new.fit(X_train_new,y_train_new)
clf_test.fit(X_train_new,y_train_new)
print('score of new method',clf_new.score(X_test_new,y_test_new))

#mimic old method
X_train=candidateDF[:m8][['climateZoneN','COOLLOAD','principleActivityN']].values
y_train=candidateDF[:m8][['MAINCLN']].values
X_test=candidateDF[m8:][['climateZoneN','COOLLOAD','principleActivityN']].values
y_test=candidateDF[m8:][['MAINCLN']].values

clf =DecisionTreeClassifier()
clf.fit(X_train,y_train)
print('score of old method',clf.score(X_test,y_test))

Load CBECS2012 successfully
dataDF in preprocessing shape (5537, 715)
top10 0.8307777777777777 top25 2.1521052631578947 top75 12.3236 top90 24.37845945945946


C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


score of new method 0.6611570247933884
score of old method 0.4791666666666667


C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\tzcha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: Data

In [129]:
CS_result1=clf.predict(X_test).reshape(X_test.shape[0],1)
X_test2=np.hstack((X_test,CS_result1))

In [130]:
np.sum(clf_test.predict(X_test2))/X_test2.shape[0]

0.42592592592592593

In [147]:
result_list=[]
for i in range(X_test.shape[0]):
    result_CL=0
    for j in range(1,5): 
        piece_data=np.hstack((X_test[i],np.array([j])))
        piece_data=piece_data.reshape(1,4)
        results=clf_new.predict(piece_data)
        if results[0]==1:
            result_CL=j
    if result_CL>0:
        result_list.append(result_CL)
    else:
        result_list.append(0)

In [148]:
result_list

[0,
 0,
 4,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 4,
 0,
 0,
 4,
 4,
 4,
 4,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 4,
 4,
 0,
 4,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 4,
 4,
 4,
 0,
 4,
 0,
 0,
 0,
 4,
 4,
 4,
 0,
 4,
 4,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 4,
 4,
 0,
 4,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 4,
 4,
 0,
 4,
 0,
 0,
 4,
 4,
 0,
 4,
 4,
 4,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 0,
 4,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 4,
 4,
 4,
 4,


In [142]:
candidateDF['MAINCLN']

0       4.0
1       2.0
3       4.0
4       4.0
5       4.0
8       2.0
9       3.0
10      2.0
11      2.0
12      1.0
14      3.0
15      3.0
17      1.0
18      3.0
19      3.0
20      4.0
21      1.0
22      2.0
23      3.0
24      1.0
25      2.0
26      4.0
27      3.0
30      4.0
31      1.0
32      2.0
33      1.0
34      2.0
35      2.0
36      1.0
       ... 
5500    3.0
5501    1.0
5502    1.0
5503    4.0
5504    1.0
5505    4.0
5506    3.0
5507    2.0
5508    4.0
5509    3.0
5510    1.0
5511    3.0
5512    3.0
5514    1.0
5515    3.0
5516    1.0
5518    4.0
5519    3.0
5522    3.0
5524    2.0
5525    3.0
5526    2.0
5528    2.0
5530    1.0
5531    2.0
5532    3.0
5533    2.0
5534    1.0
5535    3.0
5536    2.0
Name: MAINCLN, Length: 4317, dtype: float64

In [ ]:
#[candidateDF['HECS']<=1]
n=0
for i in range(X_test.shape[0]):
	CL=clf.predict(X_test[i,:3].reshape(1,3))
	if CL[0]==X_test[i,3]:
		n+=1
print('precision with test data',n/X_test.shape[0])

In [1]:
import numpy as np 
from IntelligentBuildingPerformanceDesign.AIBPD.data.database import Database
from IntelligentBuildingPerformanceDesign.AIBPD.data.building import Building
from IntelligentBuildingPerformanceDesign.AIBPD.data.preprocessing import PreprocessingCBECS
from IntelligentBuildingPerformanceDesign.AIBPD.algorithms.similarityAnalysis import Similarity
from IntelligentBuildingPerformanceDesign.AIBPD.algorithms.BN4CL import BN4CL
#select database. enbodded data base including CBECS2012.
database=Database()
CBECS2012DF=database.select('CBECS2012')
similar=Similarity()
preprocess=PreprocessingCBECS()
BN_model=BN4CL()

proposedBuilding=Building()
#define key attributes for the buildings.
proposedBuildingDict={'climateZone':1,   'principleActivity':0,
					'buildingArea': 100000, 'yearOfConstruction':1930,
					'buildingShape':2,      'wallConstruction':2,
					'WWR': 2,             'numPeople': 6,
					'COOLLOAD':11,'CDD65':1,'COOLP':2}
proposedBuilding.defineBuilding(proposedBuildingDict)
CBECS2012DF=CBECS2012DF[CBECS2012DF['CDD65']>500]
CBECS2012DF=preprocess.forHECLClf(CBECS2012DF)
CBECS2012DF=CBECS2012DF.dropna()
m=CBECS2012DF.shape[0]
m09=int(0.9*m)
similarBuildingDF=similar.subDF(CBECS2012DF)
BN_model.fit(similarBuildingDF[:m09])

Load CBECS2012 successfully
dataDF in preprocessing shape (5537, 715)
top10 0.8307777777777777 top25 2.1521052631578947 top75 12.3236 top90 24.37845945945946


C:\Users\tzcha\Documents\IntelligentBuildingPerformanceDesign\AIBPD\algorithms\BN4CL.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  similarBldDF['principleActivityN']=pd.qcut(similarBldDF['principleActivity'],5,labels=equalSizeBin5)
C:\Users\tzcha\Documents\IntelligentBuildingPerformanceDesign\AIBPD\algorithms\BN4CL.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  similarBldDF['climateZoneN']=pd.cut(similarBldDF['climateZone'],[-1,0,1,2,3,5,7],labels=[1,2,3,4,5,6])
C:\Users\tzcha\Documents\IntelligentBuildingPerformanceDesign\AIBP

BN for Main cooling equipment climateZoneDictN {1: 0.0, 2: 0.25792321566606546, 3: 0.2566348879154857, 4: 0.2404019582581809, 5: 0.00953362535429013, 6: 0.018294254058232415} COOLLOADDictN {6.0: 15, 7.0: 378, 8.0: 646, 9.0: 468, 10.0: 447, 11.0: 486, 12.0: 357, 13.0: 175, 14.0: 66} principleActivityNDictN {1: 0.18783818603452718, 2: 0.11028085544962639, 3: 0.17985055398093275, 4: 0.15691832002061323, 5: 0.147900025766555}
BN for High efficient building MAINCLDictN {1: 0.15949497552177275, 2: 0.3171862921927338, 3: 0.14944601906725072, 4: 0.1566606544704973} principleActivityNDictN {1: 0.18783818603452718, 2: 0.11028085544962639, 3: 0.17985055398093275, 4: 0.15691832002061323, 5: 0.147900025766555} CDD65NDictN {1: 0.1551146611698016, 2: 0.1545993300695697, 3: 0.1556299922700335, 4: 0.1597526410718887, 5: 0.1576913166709611} COOLPNDictN {1: 0.2896160783303272, 2: 0.49317186292192733}
Distribution of cooling system {1: 0.15949497552177275, 2: 0.3171862921927338, 3: 0.14944601906725072, 4:

In [44]:
import pandas as pd
import math
def data_preprocess_BN(similarBldDF):
		'''

		'''
		coolLoadList=[]
		equalSizeBin2=[1,2]
		equalSizeBin4=[1,2,3,4]
		equalSizeBin5=[1,2,3,4,5]
		similarBldDF['principleActivityN']=pd.qcut(similarBldDF['principleActivity'],5,labels=equalSizeBin5)
		#similarBldDF['principleActivity']#np.where(similarBldDF['principleActivity']==1, 1, 0)
		
		similarBldDF['climateZoneN']=pd.cut(similarBldDF['climateZone'],[0,1,2,3,7],labels=[1,2,3,4])
		similarBldDF['CDD65N']=pd.qcut(similarBldDF['CDD65'],5,labels=equalSizeBin5)
		similarBldDF['COOLPN']=pd.cut(similarBldDF['COOLP'],[10,90,100],labels=equalSizeBin2)
		similarBldDF['MAINCLN']=similarBldDF['MAINCL']
		similarBldDF['MAINCLN'][np.abs(similarBldDF['MAINCLN'])<1]=0
		similarBldDF['MAINCLN'][np.abs(similarBldDF['MAINCLN'])>3]=4
		#similarBldDF=similarBldDF.dropna()
		#print(similarBldDF)
		for index, row in similarBldDF.iterrows():
			if row['climateZone']==1:
				#print('climateZone')                
				coolLoadList.append(int(math.log(row['buildingArea']*0.8)))
			elif row['climateZone']==2 or row['climateZone']==5 or row['climateZone']==7:
				coolLoadList.append(int(math.log(row['buildingArea']*1.0)))
			elif row['climateZone']==3:
				coolLoadList.append(int(math.log(row['buildingArea']*1.2)))
			else:
				coolLoadList.append(int(math.log(row['buildingArea']*1.0)))
		similarBldDF['COOLLOAD']=coolLoadList
		#print('Size of similarBldDF',similarBldDF.shape,'length of cooling load',len(coolLoadList),similarBldDF['COOLLOAD'])
		del coolLoadList
		return similarBldDF[['ID','climateZoneN','COOLLOAD','principleActivityN','MAINCLN','CDD65N','COOLPN','HECS']]

In [80]:
X_test=similarBuildingDF[m09:]
X_test=X_test[X_test['HECS']==1.0]
X_test=data_preprocess_BN(X_test)

C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tzcha\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [81]:
X_test=X_test.values
n=0
for i in range(X_test.shape[0]):
    proposedBuildingList=list(X_test[i,1:7])
    proposedBuildingList.append(None)
    predictionResult=BN_model.BN4CLfitted.predict([proposedBuildingList])
    print(predictionResult[0])
    if predictionResult[0][6]==1.0:
        n+=1
print('Precision of BN = ',n/X_test.shape[0])

[3 8 5 4.0 4 1.0 0.0]
[1 9 4 4.0 2 2.0 0.0]
[2 10 3 4.0 4 1.0 0.0]
[2 10 3 4.0 3 1.0 0.0]
[1 7 4 1.0 2 2.0 0.0]
[1 7 1 1.0 4 2.0 0.0]
[1 11 4 3.0 1 1.0 0.0]
[1 8 1 4.0 1 2.0 0.0]
[2 9 3 1.0 5 2.0 0.0]
[3 8 5 4.0 5 2 0.0]
[1 9 4 4.0 2 2.0 0.0]
[1 8 4 1.0 3 1.0 0.0]
[3 8 4 2.0 3 2.0 0.0]
[2 7 1 2.0 5 2.0 0.0]
[2 9 5 4.0 5 2 0.0]
[2 9 1 2.0 4 2.0 0.0]
[1 10 1 3.0 3 1.0 0.0]
[3 9 2 4.0 5 1.0 0.0]
[3 8 4 4.0 5 2.0 0.0]
[2 11 1 1.0 5 1.0 0.0]
[1 8 1 4.0 1 1.0 0.0]
[2 11 2 2.0 5 1.0 0.0]
[3 9 1 2.0 4 1.0 0.0]
[1 10 5 2.0 3 1.0 0.0]
[1 8 3 1.0 2 1.0 0.0]
[3 9 4 2.0 4 1.0 0.0]
[2 9 1 2.0 5 2.0 0.0]
[1 9 1 2.0 1 2.0 0.0]
[3 10 2 4.0 5 1.0 0.0]
[1 9 5 2.0 1 1.0 0.0]
[3 10 3 1.0 3 1.0 1.0]
[1 8 1 4.0 2 1.0 0.0]
[2 8 4 1.0 4 2.0 0.0]
[3 11 3 2.0 5 1.0 0.0]
[1 9 1 4.0 1 2.0 0.0]
[1 7 1 4.0 1 2.0 0.0]
[1 9 4 4.0 1 2.0 0.0]
[1 8 3 4.0 2 2 0.0]
[2 8 3 1.0 5 2.0 0.0]
[1 10 5 1.0 1 1.0 0.0]
[1 10 5 4.0 2 1.0 0.0]
[2 8 5 4.0 4 2.0 0.0]
[1 8 1 4.0 2 2.0 0.0]
[1 8 3 2.0 3 1.0 0.0]
[1 12 2 3.0 4 2.0 0.0]
[2 

#### 